This weeks

In [ ]:
import os
import json
from typing import List, Dict, Any, Optional, Tuple
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from pymongo import MongoClient
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd
from docx import Document
from PyPDF2 import PdfReader
import openpyxl
import csv
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_result
# Set up Azure OpenAI environment variables
os.environ["AZURE_OPENAI_API_KEY"] = "Insert Key Here"
os.environ["AZURE_OPENAI_ENDPOINT"] = "Insert Endpoint Here"
GPT_DEPLOYMENT_NAME = "gpt-4o-mini"
API_VERSION = "2023-03-15-preview"

# MongoDB configuration
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "survey_chatbot"
COLLECTION_NAME = "survey_responses"

# Predefined survey questions <---------- This is where you add your questions! (How you want the question to be asked to the user, the actual core question)
SURVEY_QUESTIONS = [
    # Predefined survey questions
    # Core Engagement Questions
    ("How engaged did you feel during our conversation?", "Challenged me to actively engage in the discussion."),
    ("Was my explanation clear and easy to follow?", "Challenged me to understand complex ideas through clear communication."),
    ("Did you feel comfortable asking questions during our chat?", "Challenged me to ask questions and seek clarification."),
    ("Did the way I explained things help you understand the concepts better?", "Challenged me to grasp fundamental concepts."),
    ("Did our discussion encourage you to think more analytically about this?", "Challenged me to think analytically about the topic."),
    ("Was my feedback helpful for your understanding?", "Challenged me to learn from feedback and improve understanding."),

    # Learning Experience Questions
    ("Did you find our approach to this topic interesting or unique?", "Challenged me to learn through innovative approaches."),
    ("How helpful were the resources and examples I shared?", "Challenged me to learn from provided resources."),
    ("Did the different aspects of our discussion work well together?", "Challenged me to integrate different learning elements."),
    ("Did you find our back-and-forth discussion effective for learning?", "Challenged me to learn through interactive discussion."),
    ("Was it clear what we were trying to achieve in this conversation?", "Challenged me to understand learning objectives."),
    ("Did you feel like we were working together to understand this?", "Challenged me to engage in collaborative learning."),
    ("Did you feel actively involved in our discussion?", "Challenged me to participate actively."),
    ("Did my way of explaining help you understand things better?", "Challenged me to learn through varied explanations."),
    ("Did the examples I used help make things clearer?", "Challenged me to understand through practical examples."),

    # Understanding Checks
    ("Did my explanations help you know what to focus on?", "Challenged me to identify key concepts."),
    ("Could you see how things connected together?", "Challenged me to make connections between ideas."),
    ("Did you find the questions and problems challenging?", "Challenged me to tackle complex problems."),
    ("Were my explanations of what we were doing clear?", "Challenged me to follow complex processes."),
    ("Did the time we spent match the complexity of what we discussed?", "Challenged me to manage learning complexity."),
    ("Did our discussion help you achieve what you wanted to learn?", "Challenged me to achieve learning goals."),

    # Critical Thinking Questions
    ("Did you find that our discussion made you think more deeply about these concepts?", "Challenged me to think deeply about the concepts."),
    ("How did this conversation spark your creativity?", "Challenged my creativity."),
    ("Did you feel encouraged to share your thoughts during our chat?", "Challenged me to engage in discussion and debate."),
    ("How comfortable did you feel working through solutions yourself?", "Challenged me to find my own solutions to problems."),
    ("Did our conversation help you see different viewpoints on this?", "Challenged me to make decisions about different perspectives in the subject area."),
    ("Has this discussion changed how you think about this topic?", "Challenged me to reflect on my thinking to formulate a new way of seeing things."),

    # Engagement Quality Questions
    ("Did I help highlight the key points effectively?", "Challenged me to focus on essential concepts."),
    ("Did my interest in the topic come across in our discussion?", "Challenged me to engage with enthusiasm."),
    ("Did I check often enough if you were following along?", "Challenged me to monitor my own understanding."),
    ("How valuable did you find our discussion?", "Challenged me to extract value from our interaction."),
    ("Has this conversation increased your interest in the topic?", "Challenged me to develop deeper interest in the subject."),
    ("Did the discussion flow in a logical way?", "Challenged me to follow logical progressions."),
    ("Did you see how this connects to related topics?", "Challenged me to see broader connections."),
    ("Did I share any interesting current developments in this area?", "Challenged me to connect with current developments."),
    ("Did you get to practice applying what we discussed?", "Challenged me to apply concepts practically."),
    ("Did my enthusiasm for the topic help make it more interesting?", "Challenged me to engage with greater enthusiasm.")
]

# Add this helper function to check for empty responses
def is_empty_response(response: tuple) -> bool:
    """Check if the chatbot response is empty or only whitespace."""
    chatbot_response, _ = response
    return not chatbot_response or not chatbot_response.strip()
    
class DocumentManager:
    def __init__(self):
        # Store loaded documents and their query type mappings
        self.documents = {}
        self.query_document_mapping = {}
        
    def load_document(self, path: str) -> Optional[str]:
        """Load document content based on file extension"""
        try:
            ext = os.path.splitext(path)[1].lower()

            # Handle PDF files
            if ext == '.pdf':
                reader = PdfReader(path)
                return ' '.join(page.extract_text() for page in reader.pages)
            
            # Handle word files    
            elif ext == '.docx':
                doc = Document(path)
                return ' '.join(paragraph.text for paragraph in doc.paragraphs)
            
            # Handle excel files    
            elif ext == '.xlsx':
                workbook = openpyxl.load_workbook(path)
                content = []
                for sheet in workbook.sheetnames:
                    ws = workbook[sheet]
                    rows = list(ws.values)
                    content.append(f"Sheet: {sheet}")
                    content.extend([','.join(str(cell) for cell in row) for row in rows])
                return '\n'.join(content)

            # Handle CSV files
            elif ext == '.csv':
                content = []
                with open(path, 'r', encoding='utf-8') as file:
                    reader = csv.reader(file)
                    content.extend([','.join(row) for row in reader])
                return '\n'.join(content)
                
            else:
                raise ValueError(f"Unsupported file type: {ext}")
                
        except Exception as e:
            print(f"Error loading document {path}: {e}")
            return None

    def add_document(self, doc_id: str, path: str, query_types: List[str]) -> bool:
        """Add a document and its associated query types"""
        try:
            # Load the document content
            content = self.load_document(path)
            if content:
                # Store document info
                self.documents[doc_id] = {
                    'content': content,
                    'path': path,
                    'type': os.path.splitext(path)[1].lower()
                }
                
                # Map query types to this document
                for query_type in query_types:
                    if query_type not in self.query_document_mapping:
                        self.query_document_mapping[query_type] = []
                    self.query_document_mapping[query_type].append(doc_id)
                    
                return True
            return False
            
        except Exception as e:
            print(f"Error adding document: {e}")
            return False

    #In which scenario should the documents be accessed<---------------------This is where you enter in which scenario do you want which document to be accessed.
    #this category is picked by the chatbot is to connected to the query_type in the initialize_resources documents_config.the get_relevant_documents function. so the query type in that function and the categories in get_relevent_documents should match
    def get_relevant_documents(self, query: str) -> str:   
        """Get concatenated content of relevant documents based on query"""
        try:
            # Create system message for query categorization
            messages = [
                SystemMessage(content="""
                You are a helpful assistant that categorizes questions.
                Analyze the query and determine which types of information it's seeking.
                Choose from these categories:
                - schedule (course schedules, dates, times)
                - syllabus (course content,things to learn)
                - FAQ (grading policies, score calculations and other administrative details related to the course)
                - project (mini project, prpject for the course)
                
                Respond with ONLY the relevant categories as a comma-separated list.
                """),
                HumanMessage(content=f"Categorize this query: {query}")
            ]
            
            # Use Azure OpenAI to categorize the query
            llm = AzureChatOpenAI(
                temperature=0,
                deployment_name=GPT_DEPLOYMENT_NAME,
                azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                openai_api_version=API_VERSION
            )
            # Get categorization response
            response = llm.invoke(messages)
            categories = [cat.strip() for cat in response.content.split(',')]
            
            # Get relevant document contents
            relevant_docs = []
            for category in categories:
                if category in self.query_document_mapping:
                    for doc_id in self.query_document_mapping[category]:
                        if doc_id in self.documents:
                            relevant_docs.append(self.documents[doc_id]['content'])
            
            return '\n\n'.join(relevant_docs)
            
        except Exception as e:
            print(f"Error getting relevant documents: {e}")
            return ""
            
class DynamicSurveyChatbot:
    def __init__(self):
        self.user_id: Optional[str] = None  # Add user_id attribute
        self.mongo_client: Optional[MongoClient] = None
        self.db = None
        self.collection = None
        self.conversation_history: List[Any] = []
        self.conversation_turn_count: int = 0
        self.consecutive_short_responses: int = 0
        self.awaiting_survey_response: bool = False
        self.current_survey_question: Optional[Tuple[str, str]] = None
        self.available_survey_questions = list(SURVEY_QUESTIONS)
        self.asked_questions = set()
        self.response_history: Dict[str, int] = {}  # Track responses by question key
        self.document_manager = DocumentManager()
        #date time for queries regarding timing and schedule
        self.timezone = ZoneInfo("Asia/Singapore")
        self.session_id: str = datetime.now(self.timezone).strftime("%Y%m%d_%H%M%S")
        
        def get_current_datetime(self) -> datetime:
            """Get current datetime in configured timezone"""
            return datetime.now(self.timezone)
        
        self.model = AzureChatOpenAI(
            temperature=0.0,
            deployment_name=GPT_DEPLOYMENT_NAME,
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            openai_api_version=API_VERSION
        )


    
    def initialize_resources(self) -> None:
        """Initialize MongoDB connection and create indexes."""
        try:
             # Establish MongoDB connection
            self.mongo_client = MongoClient(MONGO_URI)
            self.db = self.mongo_client[DB_NAME]
            self.collection = self.db[COLLECTION_NAME]
            # Create indexes for faster querying
            self.collection.create_index([("timestamp", -1)]) # Descending order for recent items
            self.collection.create_index([("session_id", 1)])
            self.collection.create_index([("user_id", 1)])
            print("Successfully initialized MongoDB connection and indexes.")

            # Configure document sources and their associated query types <------------This is where you include your custom documents! 
            #1.add your file under 'path' 
            #2. enter a singular keyword for 'query type' and 'id'. This is a singular keyword that represents the nature of the document. Both id and query type should match
            #3.this query_type is matched to a category picked by the chatbot in the get_relevant_documents function. so the query type and the categories in get_relevent_documents should match
            documents_config = [
                {
                    'id': 'calendar',
                    'path': 'Course Calendar.csv',
                    'query_types': ['schedule']
                },
                {
                    'id': 'syllabus',
                    'path': 'SC1015_BasicInformation.pdf',
                    'query_types': ['syllabus']
                },
                {
                    'id': 'FAQ',
                    'path': 'FAQs about the Course.docx',
                    'query_types': ['FAQ']
                },
                {
                    'id': 'mini project',
                    'path': 'NTU Learn - Mini Project Page.docx',
                    'query_types': ['project']
                }
            ]
            # Load each document into the document manager
            for doc in documents_config:
                success = self.document_manager.add_document(
                    doc['id'],
                    doc['path'],
                    doc['query_types']
                )
                if success:
                    print(f"Successfully loaded document: {doc['id']}")
                else:
                    print(f"Failed to load document: {doc['id']}")
        
        except Exception as e:
            print(f"Error initializing MongoDB or resources: {e}")
            raise

    def cleanup_resources(self) -> None:
        """Clean up MongoDB connection."""
        if self.mongo_client:
            try:
                self.mongo_client.close()
                print("MongoDB connection closed successfully.")
            except Exception as e:
                print(f"Error closing MongoDB connection: {e}")

    def validate_user_id(self, user_id: str) -> bool:
        """Validate if the user ID is a 3-digit number."""
        return len(user_id) == 3 and user_id.isdigit()
        
    def save_response_to_mongodb(self, question: str, response_int: int, raw_response: str, is_update: bool = False) -> bool:
        """Save or update a survey response in MongoDB"""
        try:
            if is_update:
                #added for debugging
                print(f"Attempting to update response for question: {question}")
                print(f"New rating: {response_int}")
                print(f"User ID: {self.user_id}")
                # Update existing response
                result = self.collection.update_one(
                    {
                        "user_id": self.user_id,
                        "question": question
                    },
                    {
                        "$set": {
                            "response_value": response_int,
                            "raw_response": raw_response,
                            "updated_at": datetime.now(),
                            "is_updated": True
                        }
                    }
                )
                # Add for debugging
                print(f"Update result - matched count: {result.matched_count}, modified count: {result.modified_count}")
                success = result.modified_count > 0
            else:
                # Insert new response
                document = {
                    "session_id": self.session_id,
                    "user_id": self.user_id,
                    "timestamp": datetime.now(),
                    "question": question,
                    "response_value": response_int,
                    "raw_response": raw_response,
                    "conversation_turns": self.conversation_turn_count,
                    "conversation_history": [
                        {"role": msg.__class__.__name__, "content": msg.content}
                        for msg in self.conversation_history[-5:]# Last 5 messages
                    ]
                }
                result = self.collection.insert_one(document)
                success = bool(result.inserted_id)

            if success:
                # Update local tracking of responses
                self.response_history[question] = response_int
                print(f"Successfully {'updated' if is_update else 'saved'} survey response")
                print(f"Interpreted rating: {response_int} from response: '{raw_response}'")
                return True
            else:
                print(f"Failed to {'update' if is_update else 'save'} response: No documents were {'modified' if is_update else 'inserted'}")
                return False

        except Exception as e:
            print(f"Error {'updating' if is_update else 'saving'} response to MongoDB: {e}")
            return False


    def get_user_responses(self, user_id: str) -> List[Dict]:
        """Retrieve all survey responses for a specific user."""
        try:
            responses = list(self.collection.find(
                {"user_id": user_id},
                {
                    "question": 1,
                    "response_value": 1,
                    "timestamp": 1,
                    "raw_response": 1,  # Added to show the original response
                    "_id": 0
                }
            ))
            return responses
        except Exception as e:
            print(f"Error retrieving user responses: {e}")
            return []
    
    def check_for_rating_update(self, user_input: str) -> List[Tuple[str, str]]:
        """Check if the user is trying to update previous ratings"""
        try:
            # Get most recent question first
            most_recent = self.collection.find(
                {"user_id": self.user_id}, 
                {"question": 1}
            ).sort("timestamp", -1).limit(1)
            most_recent_list = list(most_recent)
            most_recent_question = most_recent_list[0]["question"] if most_recent_list else None
        
            # Get this user's previous responses
            user_responses = list(self.collection.find(
                {"user_id": self.user_id},
                {"question": 1, "response_value": 1, "_id": 0}
            ))
            
            # Create a list of questions this user has actually answered already
            answered_questions = [resp["question"] for resp in user_responses]
            
            messages = [
                SystemMessage(content=f"""You are a helpful assistant that determines if a user is trying to explicitly update previously answered question. 
                The user must only mention 'change', 're-rate' or 'update' their rating to a question to be considered rating update.
                Other generic responses do not count as rating update.
                If the user mentions updating or changing the rating for the 'previous question' or 'last question', use this as the question they want to update: "{most_recent_question}"
                Use semantic similarity to match questions - for example, "explanation being clear" should match "Was my explanation clear and easy to follow?"
                previously answered question:{answered_questions}
                Extract ALL rating updates the user wants to make. For each update, identify:
                1. Which question they're referring to
                2. The new rating they want to give
                Respond with ONLY a JSON object in this exact format:
                {{
                    "updates": [
                        {{
                            "question_key": string,
                            "new_response": string
                        }}
                    ],
                    "remaining_content": string or null
                }}
                """),
                HumanMessage(content=f"""
                User ID: {self.user_id}
                Questions this user has previously answered: {answered_questions}
            
                User input: {user_input}
            
                Parse this input and respond with the JSON object containing ALL rating updates.
                The question_key must be an exact match from the list of previously answered questions.
                If the user mentions "previous" or "last" question, use this most recent question: "{most_recent_question}"
                """)
            ]
            # Get LLM interpretation
            interpretation = self.model.invoke(messages)
            response_text = interpretation.content.strip()

            # Extract JSON from the response
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            if json_start >= 0 and json_end > json_start:
                json_str = response_text[json_start:json_end]
                try:
                    result = json.loads(json_str)
                    # Additional validation to ensure only previously answered questions can be updated
                    valid_updates = [
                        update for update in result["updates"]
                        if update["question_key"] in answered_questions
                    ]
                    return valid_updates, result.get("remaining_content") 
                except json.JSONDecodeError as e:
                    print(f"JSON parsing error: {e}")
                    return [], None
            return [], None

        except Exception as e:
            print(f"Error checking for rating updates: {e}")
            return [], None

    def load_user_state(self) -> None:
        """Load existing user responses when starting a new session."""
        try:
            # Get all previous responses for this user
            existing_responses = self.collection.find({"user_id": self.user_id})
            
            # Update asked_questions and response_history
            for response in existing_responses:
                self.asked_questions.add(response["question"])
                self.response_history[response["question"]] = response["response_value"]
                
            print(f"Loaded {len(self.asked_questions)} previous responses for user {self.user_id}")
        except Exception as e:
            print(f"Error loading user state: {e}")
    
    def select_survey_question(self) -> Optional[Tuple[str, str]]:
        """
        Have the LLM select the most appropriate survey question based on conversation context.
        """
        try:
            # Get recent conversation context (last few turns)
            recent_context = self.conversation_history[-10:]  # Last 5 exchanges
            context_text = "\n".join([f"{'User' if isinstance(msg, HumanMessage) else 'Assistant'}: {msg.content}" 
                                    for msg in recent_context])
            
            # Format available questions for selection - Filter out already asked questions
            available_questions = [q for q in self.available_survey_questions 
                                if q[1] not in self.asked_questions]
            
            if not available_questions:
                return None
            # Format questions for LLM prompt
            question_list = "\n".join([f"{i+1}. {q[0]}" for i, q in enumerate(available_questions)])
            #prompt for question selection
            messages = [
                SystemMessage(content="""
                You are a helpful assistant that selects the most appropriate survey question based on conversation context.
                Analyze the conversation and choose the most relevant question that matches the discussion topic and interaction style.
                Consider:
                1. The topic being discussed
                2. The depth of the conversation
                3. The user's engagement level
                4. The skills or thinking patterns demonstrated
                
                Respond only with the number of the most appropriate question.
                If no question seems appropriate, respond with 'none'.
                """),
                HumanMessage(content=f"""
                User ID: {self.user_id}
                Recent conversation:
                {context_text}
                
                Available survey questions:
                {question_list}
                
                Which question number is most appropriate to ask now?
                """)
            ]
            # Get LLM's selection
            response = self.model.invoke(messages)
            selection = response.content.strip().lower()
            
            if selection == 'none':
                return None
            # Convert selection to question index   
            try:
                question_index = int(selection) - 1
                if 0 <= question_index < len(available_questions):
                    return available_questions[question_index]
            except ValueError:
                print("Invalid question selection from LLM")
                return None

            return None

        except Exception as e:
            print(f"Error selecting survey question: {e}")
            return None
            
    def should_ask_survey_question(self, user_input: str, chatbot_response: str) -> bool:
        """Determine if it's appropriate to ask a survey question."""
        # Check if we have any questions left to ask
        if not self.available_survey_questions or len(self.asked_questions) >= len(self.available_survey_questions):
            return False
        
        # Don't interrupt if user is asking a question   
        if '?' in user_input:
            self.consecutive_short_responses = 0
            return False
        
        # Look for signals that topic is concluding    
        closure_signals = ['thanks', 'thank you', 'ok', 'got it', 'i understand', 'that makes sense']
        has_closure = any(signal in user_input.lower() for signal in closure_signals)
        
        # Track consecutive short responses to avoid interrupting ongoing discussions
        is_short_response = len(user_input.split()) <= 3
        if is_short_response:
            self.consecutive_short_responses += 1
        else:
            self.consecutive_short_responses = 0

        # Combined criteria for asking survey question
        return (has_closure and 
                self.conversation_turn_count >= 3 and 
                self.consecutive_short_responses <= 1 and
                len(chatbot_response.split()) < 100)

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=2, max=10),
        retry=retry_if_result(is_empty_response),
        before_sleep=lambda retry_state: print(f"Received empty response, retrying attempt {retry_state.attempt_number}/3...")
    )
    def generate_chatbot_response(self, user_input: str) -> Tuple[str, Optional[Tuple[str, str]]]:
        """Generate chatbot response"""
        try:
            # Get relevant document content
            doc_content = self.document_manager.get_relevant_documents(user_input)
            
            # Check for rating updates
            updates, remaining_input = self.check_for_rating_update(user_input)
            update_responses = []
            
            # Process all updates
            for update in updates:
                question_key = update["question_key"]
                new_response = update["new_response"]
                
                # First check if this question exists in the database for this user
                existing_response = self.collection.find_one({
                    "user_id": self.user_id,
                    "question": question_key
                })
            
                if existing_response:
                    # Process the update
                    response_int = self.interpret_survey_response(new_response)
                    if response_int is not None:
                        if self.save_response_to_mongodb(question_key, response_int, new_response, is_update=True):
                            update_responses.append(
                                f"I've updated your rating for the question about {question_key} to {response_int} out of 5."
                            )
                        else:
                            update_responses.append(
                                f"Sorry, I couldn't update your rating for '{question_key}'. Please try again."
                            )
                else:
                    update_responses.append(
                        f"Sorry, I couldn't find an existing rating for '{question_key}'. Please make sure you've rated this question before trying to update it."
                    )
            
            # Combine all update confirmations
            update_response = "\n".join(update_responses)
            if update_response:
                update_response += "\n\n"

            
            # If there's no remaining input after rating update, return the update confirmation
            if not remaining_input:
                return update_response.strip(), None

            # Handle ongoing survey response
            if self.awaiting_survey_response:
                survey_response = self.handle_survey_response(remaining_input)
                if update_response:
                    return update_response + survey_response, None
                return survey_response, None
        
            # Normal conversation flow
            self.conversation_turn_count += 1
            self.conversation_history.append(HumanMessage(content=user_input))

            # Get current datetime in Singapore timezone
            current_datetime = datetime.now(self.timezone)
            
            
            messages = [
                SystemMessage(content=f"""
                You are a helpful educational assistant.
                The current date and time is: {current_datetime.strftime('%Y-%m-%d %H:%M %Z')}
                
                Relevant document content:
                {doc_content}
                
                Provide clear, concise answers while maintaining a natural conversation flow.
                Focus on addressing the user's current question or topic.
                Never return an empty response.
                If you're unsure about something, say so rather than returning nothing.
                """)
            ] + self.conversation_history

            response = self.model.invoke(messages)
            chatbot_response = response.content
            self.conversation_history.append(AIMessage(content=chatbot_response))

            # Check if we should ask a survey question
            if self.should_ask_survey_question(user_input, chatbot_response):
                selected_question = self.select_survey_question()
                if selected_question:
                    self.current_survey_question = selected_question
                    self.awaiting_survey_response = True
                    full_response = update_response + f"{chatbot_response}\n\n{selected_question[0]} (Please rate from 1-5)"
                    return full_response, selected_question

            # Combine update response with chatbot response
            full_response = update_response + chatbot_response
            return full_response, None
        
        except Exception as e:
            print(f"Error generating response: {e}")
            return "I apologize, but I encountered an error. Could you please rephrase your question?", None

    def extract_survey_response(self, text: str) -> tuple[str, Optional[str]]:
        """
        Separate survey response from additional content.
        Returns (survey_part, additional_content)
        """
        try:
            # Set up system prompt for LLM to extract survey response components
            messages = [
                SystemMessage(content="""
                You are a helpful assistant that extracts survey feedback from mixed responses.
                Identify the part that answers the survey question about satisfaction/agreement.
                Respond in JSON format:
                {
                    "survey_response": "part expressing satisfaction/rating",
                    "additional_content": "remaining questions or requests if any",
                    "has_additional": true/false
                }
                If there's no clear survey response, set survey_response to null.
                """),
                HumanMessage(content=f"Extract the survey response and additional content from: '{text}'")
            ]
            # Send request to LLM and parse response
            interpretation = self.model.invoke(messages)
            result = json.loads(interpretation.content.strip())
            # Return tuple of (survey response, additional content if it exists)
            return (result.get("survey_response"), 
                   result.get("additional_content") if result.get("has_additional") else None)

        except Exception as e:
            print(f"Error extracting survey response: {e}")
            return (text, None)  # Return original text as survey response if extraction fails

    def interpret_survey_response(self, response: str) -> Optional[int]:
        """
        Use the LLM to interpret a natural language survey response into a 1-5 rating.
        """
        try:
            # Set up system prompt for LLM to convert text to numeric rating
            messages = [
                SystemMessage(content="""
                You are a helpful assistant that interprets survey responses.
                Convert the user's natural language response into a rating from 1 to 5.
                If the response contains a numeric rating (1-5), return that number. 
                otherwise,
                Treat strong affirmative responses ("yeah for sure", "definitely", "absolutely") as 5.
                Treat moderate affirmative responses ("yes", "yeah", "sure") as 4.
                Only respond with a single number from 1-5 based on the sentiment.
                5 = Very positive/strong agreement
                4 = Positive/agreement
                3 = Neutral/moderate
                2 = Negative/disagreement
                1 = Very negative/strong disagreement
                If you cannot determine a rating, respond with 'unclear'.
                Consider only the sentiment related to satisfaction/agreement.
                """),
                HumanMessage(content=f"Please convert this survey response to a rating from 1-5: '{response}'")
            ]
            # Get interpretation from LLM
            interpretation = self.model.invoke(messages)
            result = interpretation.content.strip()
            
            if result.isdigit() and 1 <= int(result) <= 5:
                return int(result)
            return None

        except Exception as e:
            print(f"Error interpreting survey response: {e}")
            return None
            
    
    def handle_survey_response(self, response: str) -> str:
        # Check if there's an active survey question
        """Process natural language survey response with improved state handling."""
        if not self.current_survey_question:
            return "No survey question is currently active."

        try:
            # Extract survey response and additional content
            survey_part, additional_content = self.extract_survey_response(response)
            
            # If no clear survey response was found, handle as regular conversation
            if not survey_part:
                # Just handle as regular conversation
                self.awaiting_survey_response = False
                return self.generate_chatbot_response(response)[0]
            
            # First check if it's a direct numerical response
            if survey_part.strip().isdigit():
                response_int = int(survey_part.strip())
                if not 1 <= response_int <= 5:
                    return "Please provide a rating from 1 to 5, or describe your experience in your own words."
            else:
                # Interpret natural language response
                response_int = self.interpret_survey_response(response)
                if response_int is None:
                    return "I couldn't clearly interpret your response. Could you please provide more detail about your experience or rate it from 1 to 5?"


            # Save survey response
            save_success = self.save_response_to_mongodb(
                question=self.current_survey_question[1],
                response_int=response_int,
                raw_response=response
            )

            if not save_success:
                return "There was an error saving your response. Please try again."

            # Update chatbot's state
            self.asked_questions.add(self.current_survey_question[1])
            self.conversation_turn_count = 0  # Reset conversation turn count
            self.awaiting_survey_response = False
            self.current_survey_question = None

            # Handle additional content if present
            if additional_content:
                follow_up_response = self.generate_chatbot_response(additional_content)[0]
                return f"I understood your rating as {response_int} out of 5. Thank you!\n\n{follow_up_response}"
            
            return f"I understood that as a {response_int} out of 5. Thank you for your feedback! How else can I help you?"

        except Exception as e:
            print(f"Error processing survey response: {e}")
            return "There was an error processing your response. Please try again."

    def run(self) -> None:
        """Main chatbot loop with improved error handling."""
        print("Welcome! You can start our conversation or type 'exit' to quit.")
        
        try:
            self.initialize_resources()

            # Get and validate user ID first
            while True:
                user_id = input("Please enter your 3-digit user ID: ").strip()
                if self.validate_user_id(user_id):
                    self.user_id = user_id
                    break
                print("Invalid user ID. Please enter a 3-digit number.")

            # Load existing state for this user
            self.load_user_state()
            
            print(f"Welcome User {self.user_id}! You can start our conversation or type 'exit' to quit.")
            print("Type 'show my responses' to see your survey responses.")

            #Main conversation loop
            while True:
                try:
                    user_input = input("\nYou: ").strip()
                    
                    # Handle exit commands
                    if user_input.lower() in ["exit", "quit", "bye"]:
                        print("Thank you for chatting! Goodbye!")
                        break

                    # Add command to view user's responses
                    if user_input.lower() == "show my responses":
                        responses = self.get_user_responses(self.user_id)
                        if responses:
                            print("\nYour survey responses:")
                            for resp in responses:
                                print(f"Question: {resp['question']}")
                                print(f"Rating: {resp['response_value']}/5")
                                print(f"Time: {resp['timestamp']}\n")
                        else:
                            print("No survey responses found.")
                        continue
                    
                    # Process input based on state - whether or not it is awaiting survey response
                    if self.awaiting_survey_response:
                        response = self.handle_survey_response(user_input)
                    else:
                        response, _ = self.generate_chatbot_response(user_input)
                        
                    # Only print response if it's not empty
                    if response and response.strip():
                        print(f"Chatbot: {response}")
                    else:
                        print("Chatbot: I apologize, but I couldn't generate a proper response even after retrying. Please try rephrasing your question.")
                    
                except KeyboardInterrupt:
                    print("\nSession interrupted. Goodbye!")
                    break
                except Exception as e:
                    print(f"Error processing input: {e}")
                    print("I apologize for the error. Please try again.")
                    
        finally:
            self.cleanup_resources()

def main():
    """Entry point with error handling."""
    try:
        chatbot = DynamicSurveyChatbot()
        chatbot.run()
    except Exception as e:
        print(f"Critical error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Welcome! You can start our conversation or type 'exit' to quit.
Successfully initialized MongoDB connection and indexes.
Successfully loaded document: calendar
Successfully loaded document: syllabus
Successfully loaded document: FAQ
Successfully loaded document: mini project


Please enter your 3-digit user ID:  123


Loaded 23 previous responses for user 123
Welcome User 123! You can start our conversation or type 'exit' to quit.
Type 'show my responses' to see your survey responses.



You:  hi


Chatbot: Hello! How can I assist you today?


In [1]:
string = "yellow"
prev = string[0]
result = []
answer = ""
count = 1  

for i in range(1, len(string)):
    if string[i] == prev:
        count += 1
    else:
        result.append(count)
        count = 1
    prev = string[i]

result.append(count)  

j = 0
for i in range(len(result)):
    j += result[i]
    answer += f"{string[j-1]}{result[i]}"

print(answer)  # Output: "y1e1l2o1w1"


y1e1l2o1w1
